In [1]:
import xml.etree.ElementTree as ET

In [2]:
def read_sym_table(fn):
    fd = ET.parse(fn)
    ret = []
    for c in fd.iter('comp'):
        if c.find('fields') is None:
            continue
        fields = list(c.iter('field'))
        fields = [(f.attrib.get("name"), f.text) for f in fields]
        fields = dict(fields)
        c = [
            c.attrib.get("ref"),
            c.find("value").text,
            c.find("footprint").text,
            fields["JLC P/N"]
        ]
        c = [f.strip() for f in c]
        ret.append(c)
    return ret

In [3]:
sym_table = read_sym_table("sman-openmv-link-pcb.xml")
sym_table = dict((l[0],l) for l in sym_table)

In [4]:
sym_table

{'D57': ['D57', 'SS14', 'Diode_SMD:D_SMA', 'C2480'],
 'F11': ['F11', '0.5A PTC', 'Fuse:Fuse_1206_3216Metric', 'C883123'],
 'C14': ['C14',
  '100uF',
  'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B',
  'C16133'],
 'C11': ['C11',
  '100uF',
  'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B',
  'C16133'],
 'C13': ['C13',
  '100uF',
  'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B',
  'C16133'],
 'C10': ['C10',
  '100uF',
  'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B',
  'C16133'],
 'C12': ['C12',
  '100uF',
  'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B',
  'C16133'],
 'C9': ['C9',
  '100uF',
  'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B',
  'C16133'],
 'C8': ['C8',
  '100uF',
  'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B',
  'C16133'],
 'D56': ['D56', '3.3V ESD', 'Diode_SMD:D_SOD-323', 'C316020'],
 'D55': ['D55', '3.3V ESD', 'Diode_SMD:D_SOD-323', 'C316020'],
 'D54': ['D54', '3.3V ESD', 'Diode_SMD:D_SOD-323', 'C316020'],
 'D53': ['D53', '3.3V ESD', 'Diode_SMD:D_SOD-323', 'C316

In [5]:
def read_cpl(fn):
    fd = open(fn).readlines()
    fd = [[f.strip() for f in l.split()] for l in fd]
    return fd

In [6]:
bom_sym_def = [
    ["Comment", 1],
    ["Designator", 0],
    ["Footprint", 2],
    ["LCSC Part #", 3]
]

In [24]:
def write_jlc_bom(fn, sym_cpl):
    with open(fn, "w") as f:
        c = [x[0] for x in bom_sym_def]
        f.write(",".join('"'+x+'"' for x in c))
        f.write("\n")
        print(sym_cpl[:10])
        # Sort by LCSC part number
        sym_cpl = sorted(sym_cpl, key=lambda x: x[3])
        l_buf = sym_cpl[0].copy()
        print(l_buf)
        # Make designator field a set
        l_buf[0] = set([l_buf[0]])
        for c in sym_cpl:
            if c[3] == l_buf[3]:
                # Add designator to set
                l_buf[0] |= set([c[0]])
            else:
                # Write line
                fields = [
                    l_buf[1],
                    ",".join(l_buf[0]),
                    l_buf[2],
                    l_buf[3]
                ]
                f.write(','.join('"{}"'.format(f) for f in fields))
                f.write('\n')
                l_buf = c.copy()
                l_buf[0] = set([l_buf[0]])
            if c is sym_cpl[-1]:
                fields = [
                    l_buf[1],
                    ",".join(l_buf[0]),
                    l_buf[2],
                    l_buf[3]
                ]
                f.write(','.join('"{}"'.format(f) for f in fields))
                f.write('\n')
                

In [12]:
def write_jlc_cpl(fn, cpl, sym_table):
    header = ["Designator", "Mid X", "Mid Y", "Layer", "Rotation"]
    with open(fn, "w") as f:
        f.write(','.join('"{}"'.format(x) for x in header))
        f.write('\n')
        for c in cpl:
            l = [cpl[c][i] for i in (0,3,4,6,5)]
            l[3] = l[3].title()
            f.write(','.join('"{}"'.format(x) for x in l))
            f.write('\n')

In [13]:
def write_jlc(subfolder, cpl_fn_rel):
    cpl = read_cpl(subfolder+"/"+cpl_fn_rel)    
    cpl = [c for c in cpl if c[0][0] != '#']
    cpl = dict([c[0], c] for c in cpl)
    sym_cpl = [sym_table[k] for k in cpl]
    write_jlc_bom(subfolder+"/jlc_bom.csv", sym_cpl)
    write_jlc_cpl(subfolder+"/jlc_cpl.csv", cpl, sym_table)

In [25]:
write_jlc("fab_openmv", "link_openmv.pos")

[['C3', '100nF', 'Capacitor_SMD:C_0402_1005Metric', 'C1525'], ['C5', '100uF', 'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B', 'C16133'], ['C6', '1uF', 'Capacitor_SMD:C_0402_1005Metric', 'C52923'], ['C7', '100nF', 'Capacitor_SMD:C_0402_1005Metric', 'C1525'], ['C8', '100uF', 'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B', 'C16133'], ['C9', '100uF', 'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B', 'C16133'], ['C10', '100uF', 'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B', 'C16133'], ['C11', '100uF', 'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B', 'C16133'], ['C12', '100uF', 'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B', 'C16133'], ['C13', '100uF', 'Capacitor_Tantalum_SMD:CP_EIA-3528-21_Kemet-B', 'C16133']]
['D2', 'Green 20mA', 'LED_SMD:LED_0603_1608Metric', 'C12624']


In [26]:
write_jlc("fab_swash", "link_swash.pos")

[['C1', '100nF', 'Capacitor_SMD:C_0402_1005Metric', 'C1525'], ['C2', '1uF', 'Capacitor_SMD:C_0402_1005Metric', 'C52923'], ['C4', '100nF', 'Capacitor_SMD:C_0402_1005Metric', 'C1525'], ['C15', '1uF', 'Capacitor_SMD:C_0402_1005Metric', 'C52923'], ['D1', '3.3V ESD', 'Diode_SMD:D_SOD-323', 'C316020'], ['D3', '5V ESD', 'Diode_SMD:D_SOD-323', 'C316020'], ['D4', '3.3V ESD', 'Diode_SMD:D_SOD-323', 'C316020'], ['D6', '3.3V ESD', 'Diode_SMD:D_SOD-323', 'C316020'], ['F1', '0.5A PTC', 'Fuse:Fuse_1206_3216Metric', 'C883123'], ['F2', '0.05A PTC', 'Fuse:Fuse_1206_3216Metric', 'C369153']]
['U3', 'SiP32431DR3', 'Package_TO_SOT_SMD:SOT-363_SC-70-6', 'C141606']
